# Rf 3 1 1_Rangeplot
Multidimensional models: projecting p.d.f and data ranges in continuous observables



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:49 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "RooProdPdf.h"
#include "RooAddPdf.h"
#include "RooPolynomial.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Create 3d pdf and data
 -------------------------------------------

Create observables

In [3]:
RooRealVar x("x", "x", -5, 5);
RooRealVar y("y", "y", -5, 5);
RooRealVar z("z", "z", -5, 5);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Create signal pdf gauss(x)*gauss(y)*gauss(z)

In [4]:
RooGaussian gx("gx", "gx", x, RooConst(0), RooConst(1));
RooGaussian gy("gy", "gy", y, RooConst(0), RooConst(1));
RooGaussian gz("gz", "gz", z, RooConst(0), RooConst(1));
RooProdPdf sig("sig", "sig", RooArgSet(gx, gy, gz));

Create background pdf poly(x)*poly(y)*poly(z)

In [5]:
RooPolynomial px("px", "px", x, RooArgSet(RooConst(-0.1), RooConst(0.004)));
RooPolynomial py("py", "py", y, RooArgSet(RooConst(0.1), RooConst(-0.004)));
RooPolynomial pz("pz", "pz", z);
RooProdPdf bkg("bkg", "bkg", RooArgSet(px, py, pz));

Create composite pdf sig+bkg

In [6]:
RooRealVar fsig("fsig", "signal fraction", 0.1, 0., 1.);
RooAddPdf model("model", "model", RooArgList(sig, bkg), fsig);

RooDataSet *data = model.generate(RooArgSet(x, y, z), 20000);

Project pdf and data on x
 -------------------------------------------------

Make plain projection of data and pdf on x observable

In [7]:
RooPlot *frame = x.frame(Title("Projection of 3D data and pdf on X"), Bins(40));
data->plotOn(frame);
model.plotOn(frame);

[#1] INFO:Plotting -- RooAbsReal::plotOn(model) plot on x integrates over variables (y,z)


Project pdf and data on x in signal range
 ----------------------------------------------------------------------------------

Define signal region in y and z observables

In [8]:
y.setRange("sigRegion", -1, 1);
z.setRange("sigRegion", -1, 1);

[#1] INFO:Eval -- RooRealVar::setRange(y) new range named 'sigRegion' created with bounds [-1,1]
[#1] INFO:Eval -- RooRealVar::setRange(z) new range named 'sigRegion' created with bounds [-1,1]


Make plot frame

In [9]:
RooPlot *frame2 = x.frame(Title("Same projection on X in signal range of (Y,Z)"), Bins(40));

Plot subset of data in which all observables are inside "sigregion"
 For observables that do not have an explicit "sigRegion" range defined (e.g. observable)
 an implicit definition is used that is identical to the full range (i.e. [-5,5] for x)

In [10]:
data->plotOn(frame2, CutRange("sigRegion"));

[#1] INFO:Plotting -- RooTreeData::plotOn: plotting 1692 events out of 20000 total events


Project model on x, integrating projected observables (y,z) only in "sigregion"

In [11]:
model.plotOn(frame2, ProjectionRange("sigRegion"));

TCanvas *c = new TCanvas("rf311_rangeplot", "rf310_rangeplot", 800, 400);
c->Divide(2);
c->cd(1);
gPad->SetLeftMargin(0.15);
frame->GetYaxis()->SetTitleOffset(1.4);
frame->Draw();
c->cd(2);
gPad->SetLeftMargin(0.15);
frame2->GetYaxis()->SetTitleOffset(1.4);
frame2->Draw();

[#1] INFO:Plotting -- RooAbsReal::plotOn(model) plot on x integrates over variables (y,z) in range sigRegion


Draw all canvases 

In [12]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()